In [32]:
from google.colab import drive
drive.mount('/content/drive')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. Imports

In [119]:
from google.colab import drive, files # google colab specific
import requests
import pandas as pd
import os
import warnings
import sys
import matplotlib.pyplot as plt
import json
import time
import numpy as np


In [34]:
sys.setrecursionlimit(10000000) # Prevent infinite recursion (https://www.geeksforgeeks.org/python-sys-setrecursionlimit-method/)

# 2. API Key
API key goes here, add your own key. <br>
Found here: https://estated.com/developers/docs/v4

In [35]:
estated_api_key = '6pc7nBaZug5BvQdv9eJMcUylvFt8Cu'

# 3. Estated Request and Response
`get_estated_detail()` implementation

Credit: https://www.youtube.com/watch?v=HxdGS48cdSU&t=506s&ab_channel=AnalyticsAriel





In [36]:
def get_estated_detail(api_key, 
                       street_address, 
                       city, 
                       state, 
                       zip_code):
  # sample API get request
  base_url = 'https://apis.estated.com/v4/property'

  params = (
      ('token', estated_api_key),
      ('street_address', street_address),
      ('city', city),
      ('state', state),
      ('zip_code', zip_code),
  )

  response = requests.get(base_url, params=params)
  return response

# 4. Create Dataframe using Address Data

In [ ]:
# Read in the addresses from the csv files and store them into a dataframe

df_addresses = pd.read_csv('/content/zillow_addresses_stPete.csv')
df_addresses = df_addresses.rename(columns={"stPete_addresses" : "address"})

# The below portion is uncessary if you manually clean the csv data before using
#********************************************************************
# clear null values
df_addresses_clean = df_addresses[df_addresses['address'].notnull()]
# drop some invalid entries
df_addresses_clean = df_addresses_clean.drop(70)
df_addresses_clean = df_addresses_clean.drop(77)
df_addresses_clean = df_addresses_clean.drop(99)
# reset index 
df_addresses_clean = df_addresses_clean.reset_index(drop=True)
#********************************************************************


In [183]:
# Manual Testing since API calls are limited and expensive

# create a dataframe
#d = {'address': ['9307 crandon lane, tampa, FL 33635', '310 72nd Ave N, Saint Petersburg, FL 33702', '135 NW Monroe Cir N, Saint Petersburg, FL 33702', '6990 18th St S, Saint Petersburg, FL 33712', '5718 18th Ave N, Saint Petersburg, FL 33710', '115 112th Ave NE APT 1027, Saint Petersburg, FL 33716', '4600 Dr Martin Luther King Jr St N, Saint Petersburg, FL 33703', '6420 6th Ave N, Saint Petersburg, FL 33710', '5386 Alhambra Way S, Saint Petersburg, FL 33712', '7403 46th Ave N LOT 274, Saint Petersburg, FL 33709', '3125 54th Ave N, Saint Petersburg, FL 33714'   ]}
#d = {'address': ['9307 crandon lane, tampa, FL 33635']}#, '310 72nd Ave N, Saint Petersburg, FL 33702']}#, '310 72nd Ave N, Saint Petersburg, FL 33702', '135 NW Monroe Cir N, Saint Petersburg, FL 33702' ]}
d = {'address': ['214 W Emily St, Tampa, FL 33603']}
_df_addresses = pd.DataFrame(data=d)
#_df_addresses

# 5. Supply Address to Dataframe
Credit: analysticsariel

In [184]:
# split dataframe into sub address elements
#df_addresses = df_addresses_clean.copy()
df_addresses = _df_addresses.copy()
df_addresses['street'] = df_addresses.apply(lambda x: x['address'].split(',')[0], axis=1)
df_addresses['city'] = df_addresses.apply(lambda x: x['address'].split(',')[1].strip(), axis=1)
df_addresses['state'] = df_addresses.apply(lambda x: x['address'].split(',')[2].split()[0], axis=1)
df_addresses['zip_code'] = df_addresses.apply(lambda x: x['address'].split(',')[2].split()[1], axis=1)
df_addresses

,address,street,city,state,zip_code
0,"214 W Emily St, Tampa, FL 33603",214 W Emily St,Tampa,FL,33603


# 6. Call `get_estated_detail()`

In [185]:
# get estated property detail per address
response_list = []

# iterate through rows in the dataframe (table)
for index, row in df_addresses.iterrows():
  
  # get parameters
  address = row['address']
  street = row['street']
  city = row['city']
  state = row['state']
  zip_code = row['zip_code']
  
  # get api response
  print('Getting data for address: {0}'.format(address))
 
  # Delay added to prevent API call bottleneck.
  time.sleep(10)
  response = get_estated_detail(estated_api_key, 
                        street, 
                        city, 
                        state, 
                        zip_code)
  # append to list
  response_list.append(response.json())

  response.json()

Getting data for address: 214 W Emily St, Tampa, FL 33603


# 7. Data Cleaning


1. Normalize required columns.
2. Drop unnneccessary columns.
3. Rename remaining columns.
4. Configure date to support monthly or yearly increments.
5. Change types to work with resampling.
6. Mitigate null values.



In [ ]:
response.json()

In [187]:
# Read each individual json response that was appended to the response list and store them in a dataframe

#for k in range(len(response_list)):
 #   df2 = df2.append(pd.json_normalize(response_list[k]["data"]["address"]))
 
# Set response
d = response.json()

# Normalize, drop unnecessary columns and rename others.
df5 = pd.json_normalize(d['data']['deeds'])
df5 = df5.drop(['document_type',
                'buyer_unit_number',
                'buyer_unit_type',
                'buyer_city',
                'buyer_address',
                'buyer2_last_name',
                'buyer2_first_name',
                'buyer_last_name',
                'buyer_first_name',
                'seller_zip_plus_four_code',
                'seller_zip_code',
                'seller_state',
                'seller_city',
                'seller_unit_number',
                'seller_address',
                'seller2_last_name',
                'seller2_first_name',
                'seller_last_name',
                'seller_first_name',
                'real_estate_owned',
                'original_contract_date',
                'deed_book',
                'deed_page',
                'document_id',
                'sale_price_description',
                'transfer_tax',
                'distressed_sale',
                'buyer_state',
                'buyer_zip_code',
                'buyer_zip_plus_four_code',
                'lender_name',
                'lender_type',
                'loan_amount',
                'loan_type',
                'loan_due_date',
                'loan_finance_type',
                'loan_interest_rate'], 
                axis = 1)
df5 = df5.rename(columns={"recording_date": "Recording Date", 
                          "sale_price": "Sale Price"})

# Change column type from object to datetime for resampling.
df5['Recording Date'] = pd.to_datetime(df5['Recording Date'])
df5['Sale Price'] = pd.to_numeric(df5['Sale Price'])

# Set index
df5 = df5.set_index('Recording Date')
df5.index = df5.index.to_period("1D")

# Resample and interpolate to fill null values.
#df5 = df5.resample('1D').interpolate()


# 7.1 Shiller Data

In [180]:
# Get Shiller data
shiller_filePath = "/content/drive/MyDrive/JPM/jphomevalue/jphomevalue/data/TPXRSA.csv"
shiller_data = pd.read_csv(shiller_filePath, header = 0)

# Convert date and set index to datetime
shiller_data['DATE'] = pd.to_datetime(shiller_data['DATE'])
shiller_data = shiller_data.set_index('DATE')

# Set period and interpolate
shiller_data.index = shiller_data.index.to_period("1D")
shiller_data = shiller_data.resample('1D').interpolate()

# 8. Initiate Automatic Interpolation Coefficient Extraction Process (A.I.C.E.P) 



In [163]:
# Create Mother Dataframe. 
# RUN ONCE ONLY!!!!!!!!!!!!!!!!

m_df = shiller_data.copy()

In [188]:
from requests.sessions import codes
# Concat the current house sale prices to the shiller index. 
pdList = [shiller_data, df5]  
new_df = pd.concat(pdList, axis = 1)

# Filter the dates where there is both Shiller Index Data and a Sale Price. 
coef_df = new_df[(new_df['TPXRSA'].notnull() & new_df['Sale Price'].notnull())]

# Create coefficient numpy array
coef_arr = []

# Compute the interpolation coefficient. 
for index, row in coef_df.iterrows():
   shiller_val = row['TPXRSA']
   house_val = row['Sale Price']
   print('Shiller Value', shiller_val)
   print('House Val', house_val)
   c = int(house_val / shiller_val)
   coef_arr.append(c)

# Compute the mean of all coefficient and that will be used as the main coefficient to interpolate the house prices based on Shiller's index
average_coef = sum(coef_arr) / len(coef_arr)

#print("The mean of all coefficients computed for that property is:", average_coef)


Shiller Value 124.4944542520049
House Val 66800.0


In [189]:
##11. Initiate Automatic Shiller Interpolation Process. 
print('The average coefficient computed is:', average_coef)
interpolated_df = new_df.copy()

# Update the Sale Prices of the House with the Interpolation Coefficient
interpolated_df['Sale Price'] = interpolated_df['TPXRSA'] * average_coef

# Clean the new interpolated dataframe of any null values which will be the entries prior to 2002. 
interpolated_df = interpolated_df[(interpolated_df['TPXRSA'].notnull() & interpolated_df['Sale Price'].notnull())]

# Now that everything is perfect append the Sale Price Column to the 'Mother Dataframe' - A separate dataframe to which the Sale Price of a House will be appended to separately. 

# Concat the new sale price column of the current house to the MDF
test_df = m_df.copy()
m_df['House 3'] = interpolated_df['Sale Price']
m_df
#interpolated_df
#interpolated_df['Sale Price'].plot()
#interpolated_df['TPXRSA'].plot()

The average coefficient computed is: 536.0


,TPXRSA,House 1,House 2,House 3
DATE,,,,
2002-01-01,120.395793,153865.823429,58833.410837,64532.145038
2002-01-02,120.405607,153878.365991,58838.206714,64537.405455
2002-01-03,120.415421,153890.908552,58843.002592,64542.665872
2002-01-04,120.425236,153903.451114,58847.798470,64547.926289
2002-01-05,120.435050,153915.993675,58852.594347,64553.186706
...,...,...,...,...
2021-11-27,321.037986,410286.545845,156880.562391,172076.360386
2021-11-28,321.231846,410534.299571,156975.295558,172180.269617
2021-11-29,321.425707,410782.053297,157070.028725,172284.178847


# Download Dataframe CSV

In [191]:
# Convert the MDF to CSV and DOWNLOAD 

m_df.to_csv('estated_output_MDF.csv')
files.download('estated_output_MDF.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>